In [ ]:
import geopandas as gpd
import pydeck as pdk
import pandas as pd
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
from geoband.API import *
import pymysql 
import pandas 
from haversine import haversine

## 버스정류장 승하차이력 데이터 합치기
GetCompasData('SBJ_2102_002', '2', '2.수원시_버스정류장별_승하차이력(1).csv')
data1=pd.read_csv('2.수원시_버스정류장별_승하차이력(1).csv')
data11=data1.groupby([data1['정류소명'], data1['정류소ID']]).sum()
data11.reset_index(inplace = True)
data11['전체']=data11['전체 승차 건수'] +data11['전체 하차 건수']
data111=data11[['정류소ID', '전체']]

GetCompasData('SBJ_2102_002', '3', '3.수원시_버스정류장별_승하차이력(2).csv')
data2=pd.read_csv('3.수원시_버스정류장별_승하차이력(2).csv')
data22=data2.groupby([data2['정류소명'], data2['정류소ID']]).sum()
data22.reset_index(inplace = True)
data22['전체']=data22['전체 승차 건수'] +data22['전체 하차 건수'] 
data222=data22[['정류소ID', '전체']]
data222.head()

GetCompasData('SBJ_2102_002', '4', '4.수원시_버스정류장별_승하차이력(3).csv')
data3=pd.read_csv( '4.수원시_버스정류장별_승하차이력(3).csv')
data33=data3.groupby([data3['정류소명'], data3['정류소ID']]).sum()
data33.reset_index(inplace = True)
data33['전체']=data33['전체 승차 건수'] +data33['전체 하차 건수'] 
data333=data33[['정류소ID', '전체']]

GetCompasData('SBJ_2102_002', '5', '5.수원시_버스정류장별_승하차이력(4).csv')
data4=pd.read_csv('5.수원시_버스정류장별_승하차이력(4).csv')
data44=data4.groupby([data4['정류소명'], data4['정류소ID']]).sum()
data44.reset_index(inplace = True)
data44['전체']=data44['전체 승차 건수'] +data44['전체 하차 건수'] 
data444=data44[['정류소ID', '전체']]

GetCompasData('SBJ_2102_002', '6', '6.수원시_버스정류장별_승하차이력(5).csv')
data5=pd.read_csv('6.수원시_버스정류장별_승하차이력(5).csv')
data55=data5.groupby([data5['정류소명'], data5['정류소ID']]).sum()
data55.reset_index(inplace = True)
data55['전체']=data55['전체 승차 건수'] +data55['전체 하차 건수'] 
data555=data55[['정류소ID', '전체']]
data555.head()

dt=pd.concat([data111,data222,data333,data444, data555] , join='inner')
dt2=dt.drop_duplicates('정류소ID')
dt2=dt.groupby([ dt['정류소ID']]).sum()
dt2.reset_index(inplace = True)

bus_locc=pd.read_csv('1.수원시_버스정류장.csv')
bus_2=bus_locc[['정류장ID','lon', 'lat']]
bus_2.dropna(axis=0, inplace = True)
bus_2.rename(columns = {'정류장ID' : '정류소ID'}, inplace = True)
bus=pd.merge(dt2, bus_2,how = 'inner', on = '정류소ID')
transfer=bus[['정류소ID','lon','lat','전체']]

## 지하철역 부근 버스정류장 확인 및 point 점수 주기
station = pd.read_csv('8.수원시_지하철역_위치정보.csv')
busstop = pd.read_csv('1.수원시_버스정류장.csv')
busstop['위치(동)'] = busstop['위치(동)'].str[8:]
bus_euc = busstop[['lon', 'lat']]
bus_euc['지하철역 부근'] = 0

# 버스정류장 근처 200m 안에 지하철역이 있으면 +1점 부여
for j in range(0,15):
    start =(station.lon[j], station.lat[j])
    for i in range(0,1179):
        goal = (bus_euc.lon[i], bus_euc.lat[i])
        distance = haversine(start, goal)
        if distance <=0.2:
            bus_euc['지하철역 부근'][i]+=1

a=pd.merge(transfer,bus_euc,how = 'right',on=['lon','lat'])

##　맛집 근처 버스정류장 확인 및 point 점수 주기
# 수원시 맛집 데이터프레임 생성
naver=pd.DataFrame(data ={'name' :['아웃백 스테이크하우스 수원역사점','길성왕꼬치','백세삼계탕','스타벅스 수원매산점','하얀풍차제과점 망포점','진미통닭','빕스 아주대점','이학순베이커리 수원점','옵스 수원점','유치회관','가보정','아웃백스테이크하우스 수원영통점','아우어베이커이 광교앨리웨이점','폴바셋 롯데몰 수원점','백세짬뽕','맥도날드 수원망포DT점','육풍 수원역점','보영만두 북문본점','삐에스몽테 제빵소','빕스 수원역사점','마키노차야 광교점','남도한정식황복촌','맥도날드 동수원GSDT점','버거킹 수원호매실SK점','본수원갈비','맥도날드 수원화성DT점','용성통닭 본점','밀도 광교점','이나경송탄부대찌개','맥도날드 수원정자DT점'],
                    'lon' : [127.000243,127.002780,126.988875,127.000719,127.056450,127.018153,127.041690,126.987894,126.997551,127.032726,127.028698, 127.073954,127.061106,126.998637,126.988933,127.054465,127.005208,127.014440,126.976586,127.000345,127.058097,127.031203,127.048343,126.961853,127.040826,127.028002,127.017591,127.061254,127.035889,127.001732], 
                    'lat' :[37.265876,37.265509,37.320279,37.320279,37.245233,37.281323,37.275928,37.323000,37.264943,37.259743,37.2743230,37.2552801,37.2741510,37.2649000,37.3182524,37.2462743,37.2689683,37.2897408,37.2451618,37.2658826,37.2873340,37.2641387,37.295094,37.266956,37.278770,37.285721,37.279724,37.274888,37.263172,37.290414]})
naver.head()

# 버스정류장 근처 200m 안에 해당 맛집이 있으면 +1점 부여
a['맛집 근처']=0
for j in naver.index:
    start =(naver.lon[j],naver.lat[j])
    for i in busstop.index:
        goal = (a.lon[i], a.lat[i])
        distance = haversine(start, goal)
        if distance <=0.2:
            a['맛집 근처'][i]+=1
      
    
a = a.rename(columns={'전체':'승하차'})
a = a.rename(columns={'정류소ID':'정류장ID'})
a = a[a['정류장ID'].isnull() == False]  # 정류장ID의 결측치 있는 행 제거
a['정류장ID'] = a['정류장ID'].astype(int)


## 버스정류장 + 고령/유소년/생산가능 격자 합치기
from shapely.geometry import Point, Polygon, LineString
from geopandas import GeoDataFrame

# 버스정류장 및 격자 데이터 불러오기
rfile = gpd.read_file('17.수원시_인구정보(고령)_격자.geojson', encoding='cp949')
child = gpd.read_file('19.수원시_인구정보(유소년)_격자.geojson', encoding='cp949')
prod = gpd.read_file('18.수원시_인구정보(생산가능)_격자.geojson', encoding='cp949')
busstop = pd.read_csv('1.수원시_버스정류장.csv')

# 버스정류장 위/경도 좌표를 Point 형식으로 변환
busstop['lon'] = busstop['lon'].astype(float)
busstop['lat'] = busstop['lat'].astype(float)
busstop['geometry'] = busstop.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
busstop = gpd.GeoDataFrame(busstop, geometry='geometry')

# 정류장 ID와 geometry 변수만 추출
bus = pd.concat([busstop.loc[:,'정류장ID'], busstop.loc[:,'geometry']], axis=1)
# 고령격자 안에 버스정류장이 있다.  
print(bus.geometry[17].within(rfile.geometry[0]))

# 고령격자 안에 버스정류장이 있으면 해당 val값 부여
bus['val'] = ''
for i in range(len(bus)):
    for j in rfile.geometry.index:
        if bus.geometry[i].within(rfile.geometry[j]) == True:
            bus.loc[i,'val'] = rfile.val[j]
        else :
            continue

#　유소년 격자 안에 버스정류장이 있으면 해당 val값 부여
bus['val_child'] = ''
for i in range(len(bus)):
    for j in child.geometry.index:
        if bus.geometry[i].within(child.geometry[j]) == True:
            bus.loc[i,'val_child'] = child.val[j]
        else :
            continue

# 생산가능 격자 안에 버스정류장이 있으면 해당 val값 부여
bus['val_prod'] = ''
for i in range(len(bus)):
    for j in prod.geometry.index:
        if bus.geometry[i].within(prod.geometry[j]) == True:
            bus.loc[i,'val_prod'] = prod.val[j]
        else :
            continue

# 버스정류장 + 고령/유소년/생산가능 격자 합친 데이터 셋 저장
bus.to_csv('bus_geometry_adult_child_prod.csv', encoding='utf-8')


## 버스정류장 + 유동인구 데이터 합치기
import pandas as pd
bus_stop = pd.read_csv('bus_geometry_adult_child_prod.csv')
bus_stop = bus_stop.iloc[:,1:]

bus_stop = pd.merge(bus_stop, busstop, on='정류장ID', how = 'left')

from geoband.API import *
GetCompasData('SBJ_2102_002', '14', '14.수원시_시간대별_유동인구(2020).csv')
human = pd.read_csv('14.수원시_시간대별_유동인구(2020).csv', encoding = 'cp949')

# 유동인구가 많은 시간대 06~08, 16~18 및 위/경도 변수 추출
time = pd.concat([human.loc[:,'TMST_06':'TMST_08'], 
human.loc[:,'TMST_16':'TMST_18'], human.loc[:, 'lon':]], axis=1)

# 위/경도를 기준으로 평균으로 그룹화
group = time.groupby(['lon','lat']).mean()
# 유동인구 많은 시간대의 평균 유동인구 파악을 위한 유동인구 평균 변수 ‘mean' 생성
group['mean'] = (group['TMST_06'] + group['TMST_07'] + group['TMST_08']+ group['TMST_16']+ group['TMST_17']+ group['TMST_18']) / 6
group = group.reset_index()

import math
# 점과 점 사이의 거리 구하는 함수
def distance(x1, y1, x2, y2):
    result = math.sqrt( math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2))
    return result

# 버스정류장과 가장 가까운 유동인구 격자점 찾기 
index_group = []
index_result = []
mini = []

for i in bus_stop.index:
    res = []
    index_bus = []
    
    for j in group.index:
        d = distance(bus_stop.loc[i,'lon'],bus_stop.loc[i,'lat'], group.loc[j,'lon'], group.loc[j,'lat'])
        res.append(d)
        index_bus.append(j)
        
    tt = min(res)
    mini.append(min(res))
    index_group.append(i)
    index_result.append(index_bus[res.index(min(res))])

bus_data = bus_stop.loc[index_group,:]
human_data = group.loc[index_result,:]
bus_data_f = bus_data.reset_index(drop=True)
human_data_f = human_data.reset_index(drop=True)

# 버스정류장 + 유동인구 데이터 합친 데이터 셋 저장
total = pd.concat([bus_data_f, human_data_f], axis=1)
total.to_csv('bus_human_data.csv', encoding='utf-8', index = False)

# 인구 데이터 셋 생성 (버스정류장 + 유동인구 + 격자)
people = pd.read_csv('bus_geometry_adult_child_prod.csv', encoding='utf-8')
people = people.iloc[:,1:]
bus_human = pd.read_csv('bus_human_data.csv', encoding='utf-8')

final_data = pd.merge(bus_human, people, on='정류장ID', how='left')
final_data.to_csv('final_data.csv', encoding = 'utf-8')

## 버스정류장별 운행노선 수 확인
import pandas as pd
final = pd.read_csv('final_data.csv', encoding='utf-8')
final = final.iloc[:,1:]

from geoband.API import *
GetCompasData('SBJ_2102_002', '7', '7.수원시_버스정류장별_노선현황.csv')
bus_line = pd.read_csv('7.수원시_버스정류장별_노선현황.csv')

# '운행노선 수' 변수 생성 및 내림차순 정렬
bus_line['split'] = bus_line['운행노선'].str.split(',')
for i in bus_line.index:
    bus_line.loc[i,'운행노선 수'] = len(bus_line.loc[i,'split'])

bus_line = bus_line.drop(columns='split')
bus_line = bus_line.sort_values(by='운행노선 수', ascending=False)

# 정류장 ID가 같으면 해당 운행노선 수 할당
final['운행노선 수'] = ''

for i in final.index:
    for j in bus_line.index:
        if final.loc[i,'정류장ID'] == bus_line.loc[j,'정류소ID']:
            final.loc[i,'운행노선 수'] = bus_line.loc[j,'운행노선 수']

final['운행노선 수'] = final['운행노선 수'].replace('', 'NaN') # 공백은 NaN으로 채우기

# 승하차 + 맛집 + 지하철역 + 유동인구 + 고령/유소년/생산가능 합친 최종 데이터 셋
data = pd.merge(final,a, on='정류장ID', how = 'left')
data.to_csv('final_final_data.csv', encoding='utf-8')

from geoband.API import *
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os

from shapely.geometry import Polygon, LineString, Point
import shapely
from tqdm import tqdm
import pydeck as pdk

os.environ["MAPBOX_API_KEY"]= 'pk.eyJ1IjoiZGFka2ltIiwiYSI6ImNrbWdqNDh3aDBiZXMydnM4NDVxb283OHYifQ.n5QzDTE7LpSJNmuBUs39AA'

GetCompasData('SBJ_2102_002', '22', '22.수원시_상세도로망_LV6.geojson')
GetCompasData('SBJ_2102_002', '23', '23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
GetCompasData('SBJ_2102_002', '24', '24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
GetCompasData('SBJ_2102_002', '25', '25.수원시_평일_일별_혼잡시간강도_LV6.csv')

road_상세 = gpd.read_file('22.수원시_상세도로망_LV6.geojson')
data_교통 = pd.read_csv('23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
data_빈도 = pd.read_csv('24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
data_시간 = pd.read_csv('25.수원시_평일_일별_혼잡시간강도_LV6.csv')

def trans(x):
    return int(str(x)[:-2])

data_교통['상세도로망_LinkID'] = data_교통['상세도로망_LinkID'].apply(trans)
data_빈도['상세도로망_LinkID'] = data_빈도['상세도로망_LinkID'].apply(trans)
data_시간['상세도로망_LinkID'] = data_시간['상세도로망_LinkID'].apply(trans)

# 도로정규화
road_상세 = pd.DataFrame(road_상세)
road_상세['width'] = road_상세['width'].astype(int)
road_상세['정규화도로폭'] = road_상세['width'] / road_상세['width'].max()
road_상세.rename(columns = {'link_id': '상세도로망_LinkID'}, inplace = True)
road_상세 = road_상세[['상세도로망_LinkID', 'road_name', 'width', 'geometry', '정규화도로폭']]
road_상세['상세도로망_LinkID'] = road_상세['상세도로망_LinkID'].astype(int)

# test = pd.merge(road_상세, data_교통, on = '상세도로망_LinkID', how = 'inner')

data_교통2 = pd.DataFrame(data_교통['전체_추정교통량'].groupby(data_교통['상세도로망_LinkID']).mean()).reset_index()
road_상세2 = pd.merge(road_상세, data_교통2, on = '상세도로망_LinkID')
data_빈도2 = pd.DataFrame(data_빈도['혼잡빈도강도'].groupby(data_빈도['상세도로망_LinkID']).mean()).reset_index()
road_상세3 = pd.merge(road_상세2, data_빈도2, on = '상세도로망_LinkID')
data_시간2 = pd.DataFrame(data_시간['혼잡시간강도'].groupby(data_시간['상세도로망_LinkID']).mean()).reset_index()
road_data = pd.merge(road_상세3, data_시간2, on = '상세도로망_LinkID')

road_data['전체_추정교통량'] = road_data['전체_추정교통량'] / road_data['전체_추정교통량'].max()
road_data['혼잡빈도강도'] = road_data['혼잡빈도강도'] / road_data['혼잡빈도강도'].max()
road_data['혼잡시간강도'] = road_data['혼잡시간강도'] / road_data['혼잡시간강도'].max()

## 버스정류장 및 인구데이터 셋 불러오기
import pandas as pd
bus = pd.read_csv('final_final_data.csv', encoding='utf-8')
bus = bus.iloc[:,1:]

# 불필요한 변수 제거
bus = bus.drop(columns=['정류장유형', '환승역타입','LED', 'LCD', 'LED+LCD복합형', '알뜰형', '임대형(음성)','lon_x','lat_x'])
bus = bus.rename(columns={'lon.1':'유동인구lon','lat.1':'유동인구lat'})
bus = bus.drop(columns=['TMST_06','TMST_07','TMST_08','TMST_16','TMST_17','TMST_18'])
#GetCompasData('SBJ_2102_002', '1', '1.수원시_버스정류장.csv')
#bus = pd.read_csv('1.수원시_버스정류장.csv')

bus['lon_y'] = bus['lon_y'].astype(float)
bus['lat_y'] = bus['lat_y'].astype(float)
bus['geometry'] = bus.apply(lambda row : Point([row['lon_y'], row['lat_y']]), axis=1)
bus = gpd.GeoDataFrame(bus, geometry='geometry')

## 버퍼 생성 및 버스정류장 할당
ax=[]
for i in road_data.index:
    ax.append(gpd.GeoSeries(road_data.geometry.iloc[i].buffer(0.0005).envelope)) # 도로근방 50m 버퍼 생성

road_data['geometry']=pd.DataFrame(ax)

data_bus = bus

from tqdm import tqdm
# 버퍼에 해당하는 범위에 속한 버스정류장에 <추정교통량 할당>
data_bus['추정교통량'] = 0.0

for i in tqdm(data_bus.index):
    for j in road_data.index:
        if data_bus.geometry[i].within(road_data.geometry[j]) == True:
            data_bus['추정교통량'][i] = road_data['전체_추정교통량'][j]
            
# 버퍼에 해당하는 범위에 속한 버스정류장에 <정규화도로폭 할당>
data_bus['정규화도로폭'] = 0.0

for i in tqdm(data_bus.index):
    for j in road_data.index:
        if data_bus.geometry[i].within(road_data.geometry[j]) == True:
            data_bus['정규화도로폭'][i] = road_data['정규화도로폭'][j]
            

# 버퍼에 해당하는 범위에 속한 버스정류장에 <혼잡빈도강도 할당>
data_bus['혼잡빈도강도'] = 0.0

for i in tqdm(data_bus.index):
    for j in road_data.index:
        if data_bus.geometry[i].within(road_data.geometry[j]) == True:
            data_bus['혼잡빈도강도'][i] = road_data['혼잡빈도강도'][j]


# 버퍼에 해당하는 범위에 속한 버스정류장에 <혼잡빈도강도 할당>
data_bus['혼잡시간강도'] = 0.0

for i in tqdm(data_bus.index):
    for j in road_data.index:
        if data_bus.geometry[i].within(road_data.geometry[j]) == True:
            data_bus['혼잡시간강도'][i] = road_data['혼잡시간강도'][j]

            
# 버스정류장 + 인구 + 대기 데이터 셋 저장 (최종 데이터 셋)           
data_bus.to_csv('data_bus.csv', encoding='utf-8')


## 버스정류장 설치 시, 인도 폭 가능한 곳 파악
# 인도 및 최종 데이터 셋 불러오기
GetCompasData('SBJ_2102_002', '26', '26.수원시_인도(2017).geojson')
road = gpd.read_file('26.수원시_인도(2017).geojson', encoding = 'utf-8')

data_bus = pd.read_csv('data_bus.csv', encoding='utf-8')
data_bus = data_bus.iloc[:,1:]

data_bus['lon_y'] = data_bus['lon_y'].astype(float)
data_bus['lat_y'] = data_bus['lat_y'].astype(float)
data_bus['geometry'] = data_bus.apply(lambda row : Point([row['lon_y'], row['lat_y']]), axis=1)
data_bus = gpd.GeoDataFrame(data_bus, geometry='geometry')

# 인도 크기만큼 buffer 생성
ax=[]

for i in road.index:
    ax.append(gpd.GeoSeries(road.geometry.iloc[i].buffer(0.00001125*road['WIDT'][i])))

road['geometry']=pd.DataFrame(ax)

# 버스정류장이 buffer에 속하면 +1점 부여
data_bus['road'] = 0

for i in data_bus.index:
    for j in road.index:
        if data_bus.geometry[i].within(road.geometry[j]) == True:
            data_bus['road'][i]+=1

# 스마트버스정류장 설치 가능 인도 폭 가진 곳 추출
data_bus_1 = data_bus[data_bus['road']>=1]
data_bus_1 = data_bus_1.drop(columns=['위치(동)'])
data_bus_1.to_csv('data_road_1.csv', encoding='utf-8')

# BIT설치여부 NaN값 및 0 제거
data_bus_1 = data_bus_1[data_bus_1['BIT설치여부'] == 1.0]
data_bus_1 = data_bus_1[data_bus_1['BIT설치여부'].isnull() == False]

## val/val_child/val_prod의 NaN값 제거
data_bus_1 = data_bus_1[data_bus_1['val'].isnull() == False]
data_bus_1 = data_bus_1[data_bus_1['val_child'].isnull() == False]
data_bus_1 = data_bus_1[data_bus_1['val_prod'].isnull() == False]
data_bus_1 = data_bus_1.reset_index(drop=True)

import geopandas as gpd
import timeit
import numpy as np
import pydeck as pdk
import os
import folium
import json
import pandas as pd
import fiona
import os

from shapely.geometry import Polygon, LineString, Point
import shapely
from tqdm import tqdm

GetCompasData('SBJ_2102_002', '11', '11.수원시_대기오염도_측정현황.csv')
GetCompasData('SBJ_2102_002', '12', '12.수원시_주차장현황.csv')
GetCompasData('SBJ_2102_002', '1', '1.수원시_버스정류장.csv')

bus = pd.read_csv('1.수원시_버스정류장.csv')
dust = pd.read_csv('11.수원시_대기오염도_측정현황.csv')
parking = pd.read_csv('12.수원시_주차장현황.csv')

# 모든 측정소에서 측정값을 보기 위해 2019.4월부터 데이터를 가져온다.
# 달마다 측정소의 측정은 1번
dust = dust.iloc[32:, :].reset_index(drop = True)

# 측정월 -> datetime으로 변환
dust['측정월'] = dust['측정월'].map(lambda x: x[:4] + '-' + x[5])
dust['측정월'] = pd.to_datetime(dust['측정월'])

# 형변환
for i in dust.columns[3:9]:
    dust[i] = dust[i].astype(float)
    
# 오염도 평균 구하기, 모든 날짜에 대한 평균
dust1 = dust[['미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)', '오존(ppm)', '이산화질소(ppm)','아황산가스(ppm)', 
      '일산화탄소(ppm)']].groupby(dust['측정소']).mean().reset_index()
dust2 = dust.loc[:, ['측정소', 'lon', 'lat']].drop_duplicates()
dust = pd.merge(dust1, dust2, on = '측정소') ## 측정소 정리 완료.

# 조사한 측정소 별 해당 동 항목
dust2 = pd.read_csv('측정소.txt')
dust2.rename(columns = {' 측정소': '측정소'}, inplace = True)
dust2['측정소'].replace('평동', '고색동', inplace = True) # 평동을 고색동으로 바꿈
dust2['행정구'] = dust2['행정구'].map(lambda x: x[8:])

### 측정소 별 해당 동 항목
dust2

##### bus 정류장에 매핑
bus['위치(동)'] = bus['위치(동)'].map(lambda x: x[8:])

dust2.rename(columns = {'행정구': '위치(동)'}, inplace = True)
#bus_측정소 = pd.merge(bus, test2, on = '위치(동)', how = 'left')
# bus_측정소 = bus_측정소[['정류장ID', '정류장명', 'lon', 'lat', '측정소']]

# top2 측정소에 해당하는 bus 정류장
#bus_측정소

dust3 = pd.merge(dust2, dust, on ='측정소', how = 'left')
data = pd.merge(bus, dust3, on ='위치(동)', how = 'left')
data = data[['정류장ID', '정류장명', '위치(동)', '미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)', 'lon_y', 'lat_y']]

# 전처리한 데이터와 대기오염 데이터 합치기
air = pd.merge(data_bus_1, data, on='정류장ID', how = 'left')
air = air.drop(columns = ['lon_y_y', 'lat_y_y',  'road', '정류장명_y',  '유동인구lon', '유동인구lat','BIT설치여부'])
data_bus_1 = air

## 공장데이터 불러오기
df = pd.read_excel("공장등록현황_20200827.xlsx")
GetCompasData('SBJ_2102_002', '31', '31.수원시_행정경계(읍면동).geojson')
geo = '31.수원시_행정경계(읍면동).geojson'
geo2 = json.load(open(geo, encoding = 'utf-8'))
dong = gpd.read_file('31.수원시_행정경계(읍면동).geojson')

df['소재지지번주소'] = df['소재지지번주소'].astype(str).map(lambda x: x[:7])
test = df[['용지면적', '건축면적', '종업원수', '공장규모구분명','대기오염등급','소재지지번주소', 'WGS84위도','WGS84경도' ]]
test.reset_index(drop = True, inplace = True)

# 변수명 변경
test.columns = ['용지면적', '건축면적', '종업원수', '공장규모구분명','대기오염등급','소재지지번주소', 'lat','lon']
test = test[test['소재지지번주소'] == '경기도 수원시']  # 수원시에 해당되는 공장만 추출
test = test.dropna()  # 결측치 제거
test = test.reset_index(drop=True)

# 공장 근처 2km 안에 버스정류장이 있으면 +1점 부여
data_bus_1['공장 근처']=0

from haversine import haversine
for j in test.index:
    start =(test.lon[j],test.lat[j])
    for i in data_bus_1.index:
        goal = (data_bus_1.lon_y_x[i], data_bus_1.lat_y_x[i])
        distance = haversine(start, goal)
        if distance <= 2:
            data_bus_1['공장 근처'][i]+=1

# 공장근처 변수 통계량 확인
data_bus_1['공장 근처'].describe()

# 사분위수를 기준으로 공장 근처변수에 대한 등급 범주화 
data_bus_1['공장 등급'] = 0

for i in data_bus_1.index:
    if data_bus_1.loc[i, '공장 근처'] < 36:
        data_bus_1.loc[i, '공장 등급'] = 1
    elif (36 <= data_bus_1.loc[i, '공장 근처']) and (data_bus_1.loc[i, '공장 근처'] <= 130):
        data_bus_1.loc[i, '공장 등급'] = 2
    else:
        data_bus_1.loc[i, '공장 등급'] = 3

# 쉘터 변수 결측치 0으로 채워주기
data_bus_1['쉘터'] = data_bus_1['쉘터'].fillna(0)
data_bus_1.isnull().sum()
data_bus_1 = data_bus_1.reset_index(drop=True)
#=======================================================================
## KMeans
# 클러스터링을 위한 셋 분리 -> 인구 데이터 클러스터링 셋 생성
human_train = data_bus_1[['mean','val', 'val_child', 'val_prod', '운행노선 수', '승하차']]

# 표준화 진행
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(human_train)
human_train = minMaxScaler.transform(human_train)
human_train = pd.DataFrame(human_train, columns = ['mean', 'val', 'val_child', 'val_prod', '운행노선 수', '승하차'])

# 클러스터링을 위한 k 수 확인
from sklearn.cluster import KMeans 
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer

model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,10))
visualizer.fit(human_train)

# K = 3 설정
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(human_train)
result_by_sklearn = human_train.copy()
result_by_sklearn["cluster"] = kmeans.labels_
result_by_sklearn
data_bus_1['cluster_h'] = result_by_sklearn['cluster']

# 시각화
import seaborn as sns
sns.scatterplot(x="lon_y_x", y="lat_y_x", hue="cluster_h", data=data_bus_1, palette="Set2");

## 주요 클러스터 그룹 선택 과정
# 인구 클러스터 그룹별 데이터 셋 생성
h_zero = data_bus_1[data_bus_1['cluster_h'] == 0]
h_one = data_bus_1[data_bus_1['cluster_h'] == 1]
h_two = data_bus_1[data_bus_1['cluster_h'] == 2]
h_zero = h_zero.drop(columns=['geometry'])
h_one = h_one.drop(columns=['geometry'])
h_two = h_two.drop(columns=['geometry'])

# 인구 클러스터링 변수 추출
h0 = h_zero.loc[:,'mean':'승하차']
h0_c = h0.columns
h1 = h_one.loc[:,'mean':'승하차']
h1_c = h1.columns
h2 = h_two.loc[:,'mean':'승하차']
h2_c = h2.columns

# 표준화 진행
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(h0)
h0 = minMaxScaler.transform(h0)
h0 = pd.DataFrame(h0)
h0.columns = h0_c
minMaxScaler.fit(h1)
h1 = minMaxScaler.transform(h1)
h1= pd.DataFrame(h1)
h1.columns = h1_c
minMaxScaler.fit(h2)
h2 = minMaxScaler.transform(h2)
h2 = pd.DataFrame(h2)
h2.columns = h2_c

# 통계량 확인
h00 = h0.describe()
h11 = h1.describe()
h22 = h2.describe()

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(x = h00.columns, y = h00.iloc[1], name = 'zero', marker_color = 'lightsalmon'))
fig.add_trace(go.Bar(x = h11.columns, y = h11.iloc[1], name = 'one', marker_color = 'indianred'))
fig.add_trace(go.Bar(x = h22.columns, y = h22.iloc[1], name = 'two', marker_color = 'green'))
fig.show()
# 대기오염 클러스터링을 위한 셋 생성(교통량 위주) - main
air_train = data_bus_1[['추정교통량','정규화도로폭', '혼잡빈도강도', '혼잡시간강도']]

# k 수 확인
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,10))
visualizer.fit(air_train)

# K = 2 설정
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(air_train)
result_by_sklearn = human_train.copy()
result_by_sklearn["cluster"] = kmeans.labels_
result_by_sklearn

data_bus_1['cluster_a'] = result_by_sklearn['cluster']
data_bus_1 # 최종 데이터셋_1

# 시각화
sns.scatterplot(x="lon_y_x", y="lat_y_x", hue="cluster_a", data=data_bus_1, palette="Set2");

# 주요 그룹 선택 진행
a_zero = data_bus_1[data_bus_1['cluster_a'] == 0]
a_one = data_bus_1[data_bus_1['cluster_a'] == 1]
a_zero = a_zero.drop(columns=['geometry'])
a_one = a_one.drop(columns=['geometry'])

a0 = a_zero.loc[:,'추정교통량':'혼잡시간강도']
a0_c = a0.columns
a1 = a_one.loc[:,'추정교통량':'혼잡시간강도']
a1_c = a1.columns

a00 = a0.describe()
a11 = a1.describe()

# 시각화
fig = go.Figure()
fig.add_trace(go.Bar(x = a00.columns, y = a00.iloc[1], name = 'zero', marker_color = 'lightsalmon'))
fig.add_trace(go.Bar(x = a11.columns, y = a11.iloc[1], name = 'one', marker_color = 'indianred'))
fig.show()

# 최종 선택된 주요그룹 -> 인구 1, 대기 1 -> 최종 데이터셋_1 생성
final = data_bus_1[data_bus_1['cluster_h'] == 1]
final = final[final['cluster_a'] == 1]
final = final.reset_index(drop=True)
final.to_csv('final_cluster_data_1.csv', encoding='utf-8')

# 대기오염 클러스터링을 위한 셋 생성(대기오염 위주) - sub
air_tr = data_bus_1[['미세먼지(㎍/㎥)','초미세먼지(㎍/㎥)', '공장 등급']]

# k 수 확인
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,10))
visualizer.fit(air_tr)

# K = 3 설정
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(air_tr)

result_by_sklearn = human_train.copy()
result_by_sklearn["cluster"] = kmeans.labels_
result_by_sklearn
data_bus_1['cluster_ar'] = result_by_sklearn['cluster']

# 시각화
sns.scatterplot(x="lon_y_x", y="lat_y_x", hue="cluster_ar", data=data_bus_1, palette="Set2");

# 주요 그룹 선택 진행
a_zero = data_bus_1[data_bus_1['cluster_ar'] == 0]
a_one = data_bus_1[data_bus_1['cluster_ar'] == 1]
a_two = data_bus_1[data_bus_1['cluster_ar'] == 2]
a_zero = a_zero.drop(columns=['geometry'])
a_one = a_one.drop(columns=['geometry'])
a_two = a_two.drop(columns=['geometry'])
a0 = a_zero.loc[:,'미세먼지(㎍/㎥)':'공장 등급']
a0_c = a0.columns
a1 = a_one.loc[:,'미세먼지(㎍/㎥)':'공장 등급']
a1_c = a1.columns
a2 = a_two.loc[:,'미세먼지(㎍/㎥)':'공장 등급']
a2_c = a2.columns

# 통계량 확인
a00 = a0.describe()
a11 = a1.describe()
a22 = a2.describe()

fig = go.Figure()
fig.add_trace(go.Bar(x = a00.columns, y = a00.iloc[1], name = 'zero', marker_color = 'lightsalmon'))
fig.add_trace(go.Bar(x = a11.columns, y = a11.iloc[1], name = 'one', marker_color = 'indianred'))
fig.add_trace(go.Bar(x = a22.columns, y = a22.iloc[1], name = 'two', marker_color = 'green'))
fig.show()

# 최종 선택된 주요그룹 -> 인구 1, 대기 1 -> 최종 데이터셋_2 생성
ff = data_bus_1[data_bus_1['cluster_h'] == 1]
ff = ff[ff['cluster_ar'] == 0]
ff = ff.reset_index(drop=True)
ff.to_csv('final_cluster_data_2.csv', encoding='utf-8')
#=======================================================================
## 버스정류장 30곳 선정 진행
# 인구데이터에서 주요 변수를 추출하여 KMeans 재진행
# 인구 데이터 주요 변수들의 상관관계 확인 (human_train)
h_corr = human_train.corr()
import seaborn as sns
sns.heatmap(data = h_corr, annot=True)
sns

# 주요 변수 추출
human_train = data_bus_1[['mean', '운행노선 수', '승하차']]
human_train

# 표준화 진행
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(human_train)
human_train = minMaxScaler.transform(human_train)
human_train = pd.DataFrame(human_train, columns = ['mean', '운행노선 수', '승하차'])

# k 수 확인
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,10))
visualizer.fit(human_train)

# K = 3 설정
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(human_train)

result_by_sklearn = human_train.copy()
result_by_sklearn["cluster"] = kmeans.labels_
result_by_sklearn
data_bus_1['cluster_h'] = result_by_sklearn['cluster']

# 시각화
sns.scatterplot(x="lon_y_x", y="lat_y_x", hue="cluster_h", data=data_bus_1, palette="Set2");

# 주요 그룹 선택
h_zero = data_bus_1[data_bus_1['cluster_h'] == 0]
h_one = data_bus_1[data_bus_1['cluster_h'] == 1]
h_two = data_bus_1[data_bus_1['cluster_h'] == 2]
h_zero = h_zero.drop(columns=['geometry'])
h_one = h_one.drop(columns=['geometry'])
h_two = h_two.drop(columns=['geometry'])

fig = go.Figure()
fig.add_trace(go.Bar(x = h00.columns, y = h00.iloc[1], name = 'zero', marker_color = 'lightsalmon'))
fig.add_trace(go.Bar(x = h11.columns, y = h11.iloc[1], name = 'one', marker_color = 'indianred'))
fig.add_trace(go.Bar(x = h22.columns, y = h22.iloc[1], name = 'two', marker_color = 'green'))
fig.show()

# 인구 2 그룹 선택
h_two = data_bus_1[data_bus_1['cluster_h'] == 2]
h_two = h_two.reset_index(drop=True)

# 지도에 시각화
import json
import folium as g
map_osm = g.Map(location=[37.2635727, 127.0286009], zoom_start=17)  # 지도 위치지정(수원시청)

rfile = open('30.수원시_법정경계(읍면동).geojson', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
g.GeoJson(jsonData, name='json_data').add_to(map_osm)

# 버스정류장 위치 마커 생성 함수
for i in range(len(h_two)):
    marker0 = g.Marker([h_two.loc[i]['lat_y_x'], h_two.loc[i]['lon_y_x']], popup = h_two.loc[i]['정류장명_x'])
    marker0.add_to(map_osm)
map_osm  

# 주요 변수 산점도 확인
set_data = h_two
sns.scatterplot(data = set_data, x='승하차', y='운행노선 수')
sns.scatterplot(data = set_data, x='승하차', y='mean')

# 데이터 셋에 ‘mean', '운행노선 수', '승하차’ 변수 표준화하여 넣어주기
human_train = set_data[['mean', '운행노선 수', '승하차']]
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(human_train)

human_train = minMaxScaler.transform(human_train)
human_train = pd.DataFrame(human_train, columns = ['mean', '운행노선 수', '승하차'])
set_data['mean'] = human_train['mean']
set_data['운행노선 수'] = human_train['운행노선 수']
set_data['승하차'] = human_train['승하차']

# 통계량 확인
set_data['운행노선 수'].describe()
set_data['승하차'].describe()
# 중앙값을 기준으로 각 변수의 값이 중앙값보다 큰 버스정류장 추출
# 승하차, 운행노선 수
line_on = set_data[set_data['승하차'] > 0.2]
line_on = line_on[line_on['운행노선 수'] > 0.6]
line_on = line_on.reset_index(drop=True)

set_data['mean'].describe()
# 승하차, mean(유동인구)
mean_on = set_data[set_data['승하차'] > 0.2]
mean_on = mean_on[mean_on['mean'] > 0.2]
mean_on = mean_on.reset_index(drop=True)

# 두 데이터 셋 합치기
data_hum = pd.concat([line_on, mean_on])
data_hum = data_hum.drop_duplicates()  # 중복 행 제거
data_hum = data_hum.reset_index(drop=True)

# 데이터 셋에 위치(동) 변수 생성
bus = pd.read_csv('1.수원시_버스정류장.csv')
bus = bus[['정류장ID','위치(동)']]
data_hum = pd.merge(data_hum, bus, on= '정류장ID', how='left')
data_hum['위치(동)'].value_counts()

# 같은 동이면 승하차 수 높은 버스정류장 하나씩만 선택
data = data_hum[['정류장ID', '위치(동)', '승하차']]

grouped = data.groupby('위치(동)')

for name, group in grouped:
    print(name)
    print(group)
    print('\n')  
# 인덱스 : 28, 43, 8, 4, 46, 31, 47, 18, 19, 15, 14, 27, 34, 21, 12, 25, 42, 35, 26, 13, 16

li = [28, 43, 8, 4, 46, 31, 47, 18, 19, 15, 14, 27, 34, 21, 12, 25, 42, 35, 26, 13, 16]
data_hum = data_hum.iloc[li,:]
data_hum = data_hum.reset_index(drop=True)
data_hum = data_hum.drop(16)
data_hum = data_hum.reset_index(drop=True)  # 총 20곳의 버스정류장 선정

# 지도에 시각화
map_osm = g.Map(location=[37.2635727, 127.0286009], zoom_start=17)  # 지도 위치지정(수원시청)

rfile = open('30.수원시_법정경계(읍면동).geojson', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
g.GeoJson(jsonData, name='json_data').add_to(map_osm)

# 버스정류장 위치 마커 생성 함수
for i in range(len(data_hum)):
    marker0 = g.Marker([data_hum.loc[i]['lat_y_x'], data_hum.loc[i]['lon_y_x']], popup = data_hum.loc[i]['정류장명_x'])
    marker0.add_to(map_osm)
map_osm  

## 나머지 10개 버스정류장 선정
# 인구2, 대기1 데이터 셋 이용(교통량 위주)
dataset = pd.read_csv('final_cluster_data_1.csv', encoding='utf-8')
dataset = dataset.iloc[:, 1:]
dataset = dataset.drop(columns = ['cluster_h', 'cluster_a', 'geometry'])
dataset['위치(동)'].value_counts()

# 앞서 선택한 버스정류장 20곳에 해당되지 않는 동의 버스정류장 추출
b1 = dataset[dataset['위치(동)'] == '광교1동']
b2 = dataset[dataset['위치(동)'] == '세류3동']
b3 = dataset[dataset['위치(동)'] == '금곡동']
b4 = dataset[dataset['위치(동)'] == '입북동']

data2 = pd.concat([b1,b2,b3,b4])
data2 = data2.drop_duplicates()
data2 = data2.reset_index(drop=True)

# 인구2, 대기1 데이터 셋 이용(대기오염 위주)
dtaset = pd.read_csv('final_cluster_data_2.csv', encoding='utf-8')
dtaset = dtaset.iloc[:, 1:]
dtaset = dtaset.drop(columns = ['cluster_ar', 'cluster_h', 'cluster_a', 'geometry'])
dtaset['위치(동)'].value_counts() 

# 앞서 선택한 버스정류장 20곳에 해당되지 않는 동의 버스정류장 추출
b1 = dtaset[dtaset['위치(동)'] == '광교1동']
b3 = dtaset[dtaset['위치(동)'] == '평동']
b4 = dtaset[dtaset['위치(동)'] == '입북동']

data3 = pd.concat([b1,b3,b4])
data3 = data3.drop_duplicates()
data3 = data3.reset_index(drop=True)

# 두 데이터셋 합치기
all_data = pd.concat([data2, data3])
all_data = all_data.drop_duplicates()  # 중복 행 제거
all_data = all_data.reset_index(drop=True)

# 같은 동이면 승하차 수 높은 버스정류장 한 곳만 선정
data = all_data[['정류장ID', '위치(동)', '승하차']]
grouped = data.groupby('위치(동)')

for name, group in grouped:
    print(name)
    print(group)
    print('\n')  # 인덱스 : 0, 4, 7, 5, 9, 13 

lis = [0, 4, 7, 5, 9, 13]
ss = all_data.iloc[lis, :]
ss = ss.reset_index(drop=True)  # 총 6곳 선정

## 최종 26곳의 버스정류장 위치 선정 및 시각화
map_osm = g.Map(location=[37.2635727, 127.0286009], zoom_start=17)  # 지도 위치지정(수원시청)

rfile = open('30.수원시_법정경계(읍면동).geojson', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
g.GeoJson(jsonData, name='json_data').add_to(map_osm)

# 버스정류장 위치 마커 생성 함수
for i in range(len(ss)):
    marker0 = g.Marker([ss.loc[i]['lat_y_x'], ss.loc[i]['lon_y_x']], popup = ss.loc[i]['정류장명_x'], icon=g.Icon(color = 'red'))
    marker0.add_to(map_osm)
    
for i in range(len(data_hum)):
    marker0 = g.Marker([data_hum.loc[i]['lat_y_x'], data_hum.loc[i]['lon_y_x']], popup = data_hum.loc[i]['정류장명_x'])
    marker0.add_to(map_osm)
map_osm  

# 변수명 변경
data_hum = data_hum.rename(columns = {'lon_y_x':'lon'})
data_hum = data_hum.rename(columns = {'lat_y_x':'lat'})
ss = ss.rename(columns = {'lon_y_x':'lon'})
ss = ss.rename(columns = {'lat_y_x':'lat'})
data_hum = data_hum.rename(columns = {'정류장명_x':'정류장명'})
ss = ss.rename(columns = {'정류장명_x':'정류장명'})
data_hum = data_hum.rename(columns = {'위치(동)_x':'위치(동)'})

In [ ]:
# 나머지 버스정류장 4곳 선정
# 맛집 근처 20개 추가 
naver=pd.DataFrame(data ={'name' :['이나경송탄부대찌개', '밀도 광교 앨리웨이', '애슐리퀸즈 수원망포점', '맥도날드 수원세류DT점', '맥도날드 수원인계DT점', '스타벅스 수원성균관대점', '버거킹 수원역사점', '스타벅스 수원천천점', '스타벅스 수원인계DT점', '광교갈비', '우판등심 수원점', '카페 노티드', '스타벅스 수원인계점', '호랑이 장가가는날', '백세삼계탕 수원고색점', '서양식탁', '스타벅스 북수원IC DT점', '수원의 아침 AK프라자수원점', '031포차 권선점', '백세삼계탕 수원조원점'],
                    'lon' : [127.035908, 127.061264, 127.061889, 127.017189, 127.035050, 126.971615, 127.000688, 126.982681, 127.0279224, 127.071672, 127.045738, 127.058073 , 127.033000, 127.019912, 126.977518, 127.0557591, 126.998390, 127.0003994, 127.01986552, 127.014295], 
                    'lat' :[37.263167, 37.274940, 37.245322,37.244761,37.270945,37.29914, 37.265961, 37.297456,37.268090,37.294808,37.248697,37.2855446,37.2658082,37.2574893, 37.2462496, 37.29452765,37.3079427, 37.2658655, 37.258283,37.296950490]})

# 맛집 근처 200m안에 버스정류장 있으면 +1
from haversine import haversine
for j in naver.index:
    start =(naver.lon[j],naver.lat[j])
    for i in dataset.index:
        goal = (dataset.lon_y_x[i], dataset.lat_y_x[i])
        distance = haversine(start, goal)
        if distance <=0.2:
            dataset['맛집 근처'][i]+=1

# 관광지 근처 500m 안에 있으면 +1
place=pd.DataFrame(data ={'name' :['수원화성', '화성행궁', '화홍문', '수원월드컵경기장', '광교호수공원', '장안문', '팔달문', '해우제', '월화원', '삼성이노베이션 뮤지엄'],
                    'lon' : [127.01201098598784, 127.01371034645877, 127.0175721388889,127.03686693282457,127.06595857607192,127.01584531594264,127.01665755235558,126.97801699424306,127.03766704079626,127.0521130407962], 
                    'lat' :[37.28819574714345, 37.283077254536536, 37.28786303904245,37.28699991918222,37.28376215114381,37.290542257439746,37.277982187263895,37.31960519702359,37.265290141197546,37.257270145810054]})

dataset['관광지 근처'] = 0
from haversine import haversine
for j in place.index:
    start =(place.lon[j],place.lat[j])
    for i in dataset.index:
        goal = (dataset.lon_y_x[i], dataset.lat_y_x[i])
        distance = haversine(start, goal)
        if distance <=0.5:
            dataset['관광지 근처'][i]+=1

# 대학가 근처 1km 안에 있으면 +1
school=pd.DataFrame(data ={'name' :['성균관대학교 자연캠퍼스' , '아주대학교', '수원대학교', '경기대학교 수원캠퍼스', '한국방송통신대학교', '수원과학대학교', '동남보건대학교', '수원여자대학교 인제캠퍼스'],
                    'lat' : [37.29426910273991,37.28215950983789,37.21142081844897,37.302824747692114,37.244951808338286, 37.196815941827786, 37.30194047826798,37.25635299318166], 
                    'lon' :[126.97553505587717,127.04367104199355,126.97658865647479,127.03602391738136,126.95715165170557,126.98312759765192,126.98604078773936,126.9629780325471]})

dataset['대학가 근처'] = 0
from haversine import haversine
for j in school.index:
    start =(school.lon[j],school.lat[j])
    for i in dataset.index:
        goal = (dataset.lon_y_x[i], dataset.lat_y_x[i])
        distance = haversine(start, goal)
        if distance <=1:
            dataset['대학가 근처'][i]+=1

# 맛집, 지하철역, 관광지, 대학가 중 하나라도 근처에 있는 버스정류장 추출
dataset_copy1 = dataset[dataset['맛집 근처'] == 1]
dataset_copy2 = dataset[dataset['지하철역 부근'] == 1]
dataset_copy3 = dataset[dataset['관광지 근처'] == 1]
dataset_copy4 = dataset[dataset['대학가 근처'] == 1]
dataset_copy5 = pd.concat([dataset_copy1,dataset_copy2,dataset_copy3,dataset_copy4])
dataset_copy5= dataset_copy5.drop_duplicates()
dataset_copy5=dataset_copy5.reset_index(drop=True)

# 4개의 버스정류장 선정
d1 = dataset[dataset['정류장명_x'] == '정자3동주민센터.정자동중심상가']
d2 = dataset[dataset['정류장명_x'] == '구운동삼환아파트']
d3 = dataset[dataset['정류장명_x'] == '뉴코아아울렛.중소기업은행']
d4 = dataset[dataset['정류장명_x'] == '살구골현대아파트.영통역4번출구']

last = pd.concat([d1, d2, d3, d4])
last = last.reset_index(drop=True)

# 변수명 변경
last = last.rename(columns = {'lon_y_x':'lon'})
last = last.rename(columns = {'lat_y_x':'lat'})
last = last.rename(columns = {'정류장명_x':'정류장명'})

# 필요 변수만 추출
last = last[['정류장ID','정류장명','lon','lat','승하차']]
data_hum = data_hum[['정류장ID','정류장명','lon','lat','승하차']]
ss = ss[['정류장ID','정류장명','lon','lat','승하차']]

human_train = last[['승하차']]
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(human_train)

human_train = minMaxScaler.transform(human_train)
human_train = pd.DataFrame(human_train, columns = ['승하차'])
last['승하차'] = human_train['승하차']

human_train = ss[['승하차']]
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(human_train)

human_train = minMaxScaler.transform(human_train)
human_train = pd.DataFrame(human_train, columns = ['승하차'])
ss['승하차'] = human_train['승하차']

final_last = pd.concat([last, ss, data_hum])
final_last = final_last.drop_duplicates()
final_last = final_last.reset_index(drop=True)

# 우선순위 선정
one = final_last.iloc[10:,:]
two = final_last.iloc[4:10, :]
three = final_last.iloc[:4, :]

one = one.sort_values('승하차', ascending=False)
two = two.sort_values('승하차', ascending=False)
three = three.sort_values('승하차', ascending=False)
final_last = pd.concat([one,two,three])
final_last = final_last.reset_index(drop=True)
final_last.to_csv('last_final_dataset_20210331.csv', encoding='utf-8')

# 최종 30곳의 버스정류장 시각화
map_osm = g.Map(location=[37.2635727, 127.0286009], zoom_start=17)  # 지도 위치지정(수원시청)

rfile = open('30.수원시_법정경계(읍면동).geojson', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
g.GeoJson(jsonData, name='json_data').add_to(map_osm)

# 버스정류장 위치 마커 생성 함수
for i in range(len(ss)):
    marker0 = g.Marker([ss.loc[i]['lat'], ss.loc[i]['lon']], popup = ss.loc[i]['정류장명'], icon=g.Icon(color = 'red'))
    marker0.add_to(map_osm)
    
for i in range(len(data_hum)):
    marker0 = g.Marker([data_hum.loc[i]['lat'], data_hum.loc[i]['lon']], popup = data_hum.loc[i]['정류장명'], icon=g.Icon(color = 'orange'))
    marker0.add_to(map_osm)
    
for i in range(len(last)):
    marker0 = g.Marker([last.loc[i]['lat'], last.loc[i]['lon']], popup = last.loc[i]['정류장명'], icon=g.Icon(color = 'black'))
    marker0.add_to(map_osm)
map_osm  

In [ ]:
## 광고 타겟층 선정
from geoband.API import *
import pandas as pd
GetCompasData('SBJ_2102_002', '15', '15.수원시_성연령별_유동인구(2020).csv')
sexage = pd.read_csv('15.수원시_성연령별_유동인구(2020).csv')

data = pd.read_csv('last_final_dataset_20210331.csv', encoding='utf-8')
data = data.iloc[:,1:]

# 위도, 경도를 기준으로 평균으로 그룹화
sexage_group = sexage.groupby(['lon','lat']).mean()
sexage_group.reset_index(inplace = True)
sexage_group.iloc[:,:2]
group=pd.concat([sexage_group.iloc[:,:2], sexage_group.iloc[:,3:]], axis=1)

# 구분을 위한 변수명 변경
group = group.rename(columns = {'lon':'g_lon'})
group = group.rename(columns = {'lat':'g_lat'})

import math
# 점과 점 사이의 거리 구하는 함수
def distance(x1, y1, x2, y2):
    result = math.sqrt( math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2))
    return result

# 유동인구 격자 중심점과 거리가 가장 가까운 버스정류장 추출 
index_group = []
index_result = []
mini = []

for i in data.index:
    res = []
    index_bus = []
    
    for j in group.index:
        d = distance(data.loc[i,'lon'],data.loc[i,'lat'], group.loc[j,'g_lon'], group.loc[j,'g_lat'])
        res.append(d)
        index_bus.append(j)
        
    tt = min(res)
    mini.append(min(res))
    index_group.append(i)
    index_result.append(index_bus[res.index(min(res))])

bus_data = data.loc[index_group,:]
human_data = group.loc[index_result,:]
bus_data_f = bus_data.reset_index(drop=True)
human_data_f = human_data.reset_index(drop=True)

# 유동인구 + 버스정류장 데이터 셋
total = pd.concat([bus_data_f, human_data_f], axis=1)


# 행에서 가장 큰 값을 갖는 열 추출
a = total['정류장ID']
b = total['정류장명']
lon = total['lon']
lat = total['lat']
total= total.drop(columns=['정류장ID','정류장명','lon','lat','g_lon','g_lat'])

ab = total.idxmax(axis=1)

total['정류장ID'] = a
total['정류장명'] = b 
total['나이성별'] = ab
total['lon'] = lon
total['lat'] = lat

total = total[['정류장명', '정류장ID', 'lon', 'lat', '나이성별']]
total = total.rename(columns={'나이성별':'광고Target층'})

for i in total.index:
    if total.loc[i, '광고Target층'] == 'MAN_FLOW_POP_CNT_60GU':
        total.loc[i, '광고Target층'] = '60대 남성'
    elif total.loc[i, '광고Target층'] == 'MAN_FLOW_POP_CNT_30G':
        total.loc[i, '광고Target층'] = '30대 남성'
    elif total.loc[i, '광고Target층'] == 'MAN_FLOW_POP_CNT_40G':
        total.loc[i, '광고Target층'] = '40대 남성'
    elif total.loc[i, '광고Target층'] == 'MAN_FLOW_POP_CNT_50G':
        total.loc[i, '광고Target층'] = '50대 남성'
    elif total.loc[i, '광고Target층'] == 'WMAN_FLOW_POP_CNT_20G':
        total.loc[i, '광고Target층'] = '20대 여성'
    elif total.loc[i, '광고Target층'] == 'MAN_FLOW_POP_CNT_20G':
        total.loc[i, '광고Target층'] = '20대 남성'
        
total.to_csv('분석결과.csv', encoding='utf-8')